# Data-Driven Pricing Strategy – Analysis Workbook

This notebook documents the end-to-end analysis used to build the pricing framework. It aligns directly with the Final Assessment Report and supports all pricing decisions with reproducible analysis and visuals.

## 1. Data Loading & Preparation
Load and merge pricing, sales, inventory, advertising, and competitor datasets at SKU level.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pricing = pd.read_csv('Pricing_Data.csv')
sales = pd.read_csv('Historical_Sales.csv')
inventory = pd.read_csv('Inventory_Health.csv')
ads = pd.read_csv('Ads_Performance.csv')
competitor = pd.read_csv('Competitor_Data.csv')

for df in [pricing, sales, inventory, ads, competitor]:
    if 'SKU' not in df.columns:
        df.rename(columns={df.columns[0]: 'SKU'}, inplace=True)

df = pricing.merge(sales, on='SKU', how='left') \
           .merge(inventory, on='SKU', how='left') \
           .merge(ads, on='SKU', how='left') \
           .merge(competitor, on='SKU', how='left')

df.head()

## 2. KPI Creation
Derive profitability, demand, and efficiency KPIs to drive pricing decisions.

In [ ]:
df['Current_Price'] = df['Current_Price'].replace('[\$,]', '', regex=True).astype(float)
df['Cost'] = df['Cost'].replace('[\$,]', '', regex=True).astype(float)
df['FBA Fee'] = df['FBA Fee'].replace('[\$,]', '', regex=True).astype(float)

df['Gross_Margin'] = df['Current_Price'] - (df['Cost'] + df['FBA Fee'])
df['Margin_%'] = (df['Gross_Margin'] / df['Current_Price']) * 100

df[['SKU', 'Current_Price', 'Gross_Margin', 'Margin_%']].head()

## 3. Profitability Analysis
Visualize margin distribution to identify margin leakage.

In [ ]:
plt.figure()
df['Margin_%'].dropna().hist()
plt.title('Margin % Distribution Across SKUs')
plt.xlabel('Margin %')
plt.ylabel('SKU Count')
plt.show()

## 4. Advertising Efficiency Analysis
Analyze the relationship between ACOS and margins.

In [ ]:
plt.figure()
plt.scatter(df['acosClicks7d'], df['Margin_%'])
plt.xlabel('ACOS %')
plt.ylabel('Margin %')
plt.title('ACOS vs Margin')
plt.show()

## 5. Inventory Risk Analysis
Use pricing as a lever to manage overstock and stockout risk.

In [ ]:
inv_col = next((c for c in df.columns if 'inventory' in c.lower()), None)
if inv_col:
    plt.figure()
    plt.scatter(df[inv_col], df['sales30d'])
    plt.xlabel('Inventory Level')
    plt.ylabel('30 Day Sales')
    plt.title('Inventory vs Sales Velocity')
    plt.show()

## 6. Pricing Logic & Recommendations
Apply rule-based logic to generate recommended prices.

In [ ]:
def recommend_price(row):
    price = row['Current_Price']
    if row['Margin_%'] < 20:
        return round(price * 1.07, 2)
    if row.get('sales30d', 0) < 10:
        return round(price * 0.93, 2)
    return round(price, 2)

df['Recommended_Price'] = df.apply(recommend_price, axis=1)
df[['SKU', 'Current_Price', 'Recommended_Price']].head(10)

## 7. Summary
This notebook provides transparent, reproducible support for the pricing framework presented in the final assessment report.